## CalenderAI using HugginFace

- Test Introduction

In [44]:
import os
import dotenv
from huggingface_hub import InferenceClient
client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct", token=os.getenv("Hugging_Face_Browser_API_KEY"))


completion = client.chat.completions.create(
    model="ollama",
    messages=[
        {"role": "system", "content": "You're a helpful assistant."},
        {
            "role": "user",
            "content": "Write a limerick about the Python programming language.",
        },
    ],
)

response = completion.choices[0].message.content
print(response)


There once was a Python so bright,
Its syntax was easy to write.
It handled with flair,
Modules beyond compare,
And its libraries shone with great light.


- Test Routing

In [45]:
from typing import Optional, Literal
from pydantic import BaseModel, Field
from huggingface_hub import InferenceClient
import os
import logging
import json

In [3]:

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)


In [63]:
client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct", token=os.getenv("Hugging_Face_Browser_API_KEY"))
model="ollama"


In [201]:
CalendarRequestType.model_json_schema()

{'description': 'Router LLM call: Determine the type of calendar request',
 'properties': {'request_type': {'description': 'Type of calendar request being made',
   'enum': ['new_event', 'modify_event', 'other'],
   'title': 'Request Type',
   'type': 'string'},
  'confidence_score': {'description': 'Confidence score number between 0 and 1',
   'maximum': 1.0,
   'minimum': 0.0,
   'title': 'Confidence Score',
   'type': 'number'},
  'description': {'description': 'Cleaned description of the request',
   'title': 'Description',
   'type': 'string'}},
 'required': ['request_type', 'confidence_score', 'description'],
 'title': 'CalendarRequestType',
 'type': 'object'}

In [214]:

# --------------------------------------------------------------
# Step 1: Define the data models for routing and responses
# --------------------------------------------------------------

class CalendarRequestType(BaseModel):
    """Router LLM call: Determine the type of calendar request"""

    request_type: Literal["new_event", "modify_event", "other"] = Field(
        description="Type of calendar request being made"
    )
    confidence_score: float = Field(description="Confidence score is a number between 0.0 and 1.0")
                     
    description: str = Field(description="Cleaned description of the request")
    
 

class NewEventDetails(BaseModel):
    """Details for creating a new event"""

    name: str = Field(description="Name of the event")
    date: str = Field(description="Date and time of the event (ISO 8601)")
    duration_minutes: int = Field(description="Duration in minutes")
    participants: list[str] = Field(description="List of participants")


class Change(BaseModel):
    """Details for changing an existing event"""

    field: str = Field(description="Field to change")
    new_value: str = Field(description="New value for the field")


class ModifyEventDetails(BaseModel):
    """Details for modifying an existing event"""

    event_identifier: str = Field(
        description="Description to identify the existing event"
    )
    changes: list[Change] = Field(description="List of changes to make")
    participants_to_add: list[str] = Field(description="New participants to add")
    participants_to_remove: list[str] = Field(description="Participants to remove")


class CalendarResponse(BaseModel):
    """Final response format"""

    success: bool = Field(description="Whether the operation was successful")
    message: str = Field(description="User-friendly response message")
    calendar_link: Optional[str] = Field(description="Calendar link if applicable")



In [202]:

# --------------------------------------------------------------
# Step 2: Define the routing and processing functions
# --------------------------------------------------------------


def route_calendar_request(user_input: str) -> CalendarRequestType:
    """Router LLM call to determine the type of calendar request"""
    logger.info("Routing calendar request")
    
    #-----add the schema to the response format   
    response_format = {
    "type": "json",
    "value": CalendarRequestType.model_json_schema()  # Add the generated schema as the response format
    }
    
    #-----write the messages to pass to the model
    messages=[
            {
                "role": "system",
                "content": "Determine if this is a request to create a new calendar event or modify an existing one.",
            },
            {"role": "user", "content": user_input},
    ]

    completion = client.chat.completions.create(
        model=model,
        messages= messages,
        response_format=response_format
    )
    
    result = completion.choices[0].message.content
    result=json.loads(result)
    calednType=CalendarRequestType(**result)
    #.parsed
    # result_dict = json.loads(result)
    # print("result_dict", result_dict)
    # data=json.loads(result_dict)
    # print('data', data)
    # test = CalendarRequestType(**data)
    logger.info(
        f"Request routed as: {calednType}",
       # f"Request routed as: {test.request_type} with confidence: {test.confidence_score}"
    )
    return calednType


def handle_new_event(description: str) -> CalendarResponse:
    """Process a new event request"""
    logger.info("Processing new event request")

    # Get event details
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "Extract details for creating a new calendar event.",
            },
            {"role": "user", "content": description},
        ],
        response_format=NewEventDetails,
    )
    details = completion.choices[0].message.parsed

    logger.info(f"New event: {details.model_dump_json(indent=2)}")

    # Generate response
    return CalendarResponse(
        success=True,
        message=f"Created new event '{details.name}' for {details.date} with {', '.join(details.participants)}",
        calendar_link=f"calendar://new?event={details.name}",
    )


def handle_modify_event(description: str) -> CalendarResponse:
    """Process an event modification request"""
    logger.info("Processing event modification request")

    # Get modification details
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "Extract details for modifying an existing calendar event.",
            },
            {"role": "user", "content": description},
        ],
        response_format=ModifyEventDetails,
    )
    details = completion.choices[0].message.parsed

    logger.info(f"Modified event: {details.model_dump_json(indent=2)}")

    # Generate response
    return CalendarResponse(
        success=True,
        message=f"Modified event '{details.event_identifier}' with the requested changes",
        calendar_link=f"calendar://modify?event={details.event_identifier}",
    )


def process_calendar_request(user_input: str) -> Optional[CalendarResponse]:
    """Main function implementing the routing workflow"""
    logger.info("Processing calendar request")

    # Route the request
    route_result = route_calendar_request(user_input)

    # # Check confidence threshold
    # if route_result.confidence_score < 0.7:
    #     logger.warning(f"Low confidence score: {route_result.confidence_score}")
    #     return None

    # # Route to appropriate handler
    # if route_result.request_type == "new_event":
    #     return handle_new_event(route_result.description)
    # elif route_result.request_type == "modify_event":
    #     return handle_modify_event(route_result.description)
    # else:
    #     logger.warning("Request type not supported")
    #     return None
    return route_result


In [215]:

# --------------------------------------------------------------
# Step 3: Test with new event
# --------------------------------------------------------------

new_event_input = "let's schedule a meeting with John and Jane on Monday at 10 AM"
result = process_calendar_request(new_event_input)
if result:
    print(f"Response: {result}")


2025-02-14 11:03:16 - INFO - Processing calendar request
2025-02-14 11:03:16 - INFO - Routing calendar request
2025-02-14 11:03:17 - INFO - Request routed as: request_type='new_event' confidence_score=96.0 description='meeting with John and Jane on Monday at 10 AM'


Response: request_type='new_event' confidence_score=96.0 description='meeting with John and Jane on Monday at 10 AM'


In [142]:
CalendarRequestType.model_json_schema()

{'description': 'Router LLM call: Determine the type of calendar request',
 'properties': {'request_type': {'description': 'Type of calendar request being made',
   'enum': ['new_event', 'modify_event', 'other'],
   'title': 'Request Type',
   'type': 'string'},
  'confidence_score': {'description': 'Confidence score between 0 and 1',
   'title': 'Confidence Score',
   'type': 'number'},
  'description': {'description': 'Cleaned description of the request',
   'title': 'Description',
   'type': 'string'}},
 'required': ['request_type', 'confidence_score', 'description'],
 'title': 'CalendarRequestType',
 'type': 'object'}

In [143]:
CalendarRequestType.schema()

/tmp/ipykernel_35516/3517091682.py:1: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  CalendarRequestType.schema()


{'description': 'Router LLM call: Determine the type of calendar request',
 'properties': {'request_type': {'description': 'Type of calendar request being made',
   'enum': ['new_event', 'modify_event', 'other'],
   'title': 'Request Type',
   'type': 'string'},
  'confidence_score': {'description': 'Confidence score between 0 and 1',
   'title': 'Confidence Score',
   'type': 'number'},
  'description': {'description': 'Cleaned description of the request',
   'title': 'Description',
   'type': 'string'}},
 'required': ['request_type', 'confidence_score', 'description'],
 'title': 'CalendarRequestType',
 'type': 'object'}

In [ ]:

# --------------------------------------------------------------
# Step 4: Test with modify event
# --------------------------------------------------------------

modify_event_input = (
    "Can you move the team meeting with Alice and Bob to Wednesday at 3pm instead?"
)
result = process_calendar_request(modify_event_input)
if result:
    print(f"Response: {result.message}")


In [ ]:

# --------------------------------------------------------------
# Step 5: Test with invalid request
# --------------------------------------------------------------

invalid_input = "What's the weather like today?"
result = process_calendar_request(invalid_input)
if not result:
    print("Request not recognized as a calendar operation")


In [85]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key=os.getenv("Hugging_Face_Browser_API_KEY"))

messages = [
	{
		"role": "user",
		"content": "What is the capital of France?"
	}
]

stream = client.chat.completions.create(
    model="google/gemma-2-2b-it", 
	messages=messages, 
	max_tokens=500,
	stream=True,
)

for chunk in stream:
    print(chunk)

ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content='The', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1739523105, id='', model='google/gemma-2-2b-it', system_fingerprint='3.0.1-sha-bb9095a', usage=None)
ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content=' capital', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1739523105, id='', model='google/gemma-2-2b-it', system_fingerprint='3.0.1-sha-bb9095a', usage=None)
ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role='assistant', content=' of', tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1739523105, id='', model='google/gemma-2-2b-it', system_fingerprint='3.0.1-sha-bb9095a', usage=None)
ChatCompletionStreamOutput(choices=[ChatCompletionStream

In [87]:
from huggingface_hub import InferenceClient
client = InferenceClient("meta-llama/Llama-3.2-3B-Instruct",api_key=os.getenv("Hugging_Face_Browser_API_KEY"))
messages = [
    {
        "role": "user",
        "content": "I saw a puppy a cat and a raccoon during my bike ride in the park. What did I saw and when?",
    },
]
response_format = {
    "type": "json",
    "value": {
        "properties": {
            "location": {"type": "string"},
            "activity": {"type": "string"},
            "animals_seen": {"type": "integer", "minimum": 1, "maximum": 5},
            "animals": {"type": "array", "items": {"type": "string"}},
        },
        "required": ["location", "activity", "animals_seen", "animals"],
    },
}
response = client.chat_completion(
    messages=messages,
    response_format=response_format,
    max_tokens=500,
)
response.choices[0].message.content

'{ "location":"park", "activity":"bike ride", "animals_seen": 3, "animals": ["puppy", "cat", "raccoon"] }'